<a href="https://colab.research.google.com/github/basmadokkar/CV/blob/main/UNet_image_segmentation_Oxford_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
import glob
import os
import cv2
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda,Dense, Activation, Concatenate, MaxPool2D
from keras import optimizers
from keras.optimizers import Adam
from keras.metrics import MeanIoU
from keras import backend as K
from sklearn import metrics
from google.colab import drive
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from skimage.filters import threshold_otsu, threshold_multiotsu
from cv2 import adaptiveThreshold
from PIL import Image

In [ ]:

def binary_crossentropy(Y_pred, Y_gt):
    epsilon = 1.e-5
    Y_pred = tf.clip_by_value(Y_pred, epsilon, 1. - epsilon)
    logits = tf.log(Y_pred / (1 - Y_pred))
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_gt, logits=logits)
    loss = tf.reduce_mean(loss)
    return loss


def dice_loss_3d(Y_gt, Y_pred):
    Z, H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    pred_flat = tf.reshape(Y_pred, [-1, H * W * C * Z])
    true_flat = tf.reshape(Y_gt, [-1, H * W * C * Z])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + smooth
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + smooth
    loss = 1 - tf.reduce_mean(intersection / denominator)
    return loss


def dice_loss_2d(Y_gt, Y_pred):
    H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    pred_flat = tf.reshape(Y_pred, [-1, H * W * C])
    true_flat = tf.reshape(Y_gt, [-1, H * W * C])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + smooth
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + smooth
    loss = 1 - tf.reduce_mean(intersection / denominator)
    return loss


def tversky_loss_3d(Y_gt, Y_pred, alpha=0.7):
    Z, H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    y_pred_pos = tf.reshape(Y_pred, [-1, H * W * C * Z])
    y_true_pos = tf.reshape(Y_gt, [-1, H * W * C * Z])
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos, axis=1)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos), axis=1)
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos, axis=1)
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)
    loss = 1 - tf.reduce_mean(tversky)
    return loss


def tversky_loss_2d(Y_gt, Y_pred, alpha=0.7):
    H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    y_pred_pos = tf.reshape(Y_pred, [-1, H * W * C])
    y_true_pos = tf.reshape(Y_gt, [-1, H * W * C])
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos, axis=1)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos), axis=1)
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos, axis=1)
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)
    loss = 1 - tf.reduce_mean(tversky)
    return loss


def focal_tversky_3d(Y_gt, Y_pred, alpha=0.7, gamma=0.75):
    Z, H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    y_pred_pos = tf.reshape(Y_pred, [-1, H * W * C * Z])
    y_true_pos = tf.reshape(Y_gt, [-1, H * W * C * Z])
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos, axis=1)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos), axis=1)
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos, axis=1)
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)
    loss = 1 - tf.reduce_mean(tversky)
    loss = tf.pow(loss, gamma)
    return loss


def focal_tversky_2d(Y_gt, Y_pred, alpha=0.7, gamma=0.75):
    H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    y_pred_pos = tf.reshape(Y_pred, [-1, 3*256*256])
    y_true_pos = tf.reshape(Y_gt, [-1, 3*256*256])
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos, axis=1)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos), axis=1)
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos, axis=1)
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)
    loss = 1 - tf.reduce_mean(tversky)
    loss = tf.pow(loss, gamma)
    return loss


def generalised_dice_loss_3d(Y_gt, Y_pred):
    smooth = 1e-5
    w = tf.reduce_sum(Y_gt, axis=[1, 2, 3])
    w = 1 / (w ** 2 + smooth)

    numerator = Y_gt * Y_pred
    numerator = w * tf.reduce_sum(numerator, axis=[1, 2, 3])
    numerator = tf.reduce_sum(numerator, axis=1)

    denominator = Y_pred + Y_gt
    denominator = w * tf.reduce_sum(denominator, axis=[1, 2, 3])
    denominator = tf.reduce_sum(denominator, axis=1)

    gen_dice_coef = 2 * numerator / (denominator + smooth)
    loss = tf.reduce_mean(1 - gen_dice_coef)
    return loss


def generalised_dice_loss_2d_ein(Y_gt, Y_pred):
    Y_gt = tf.cast(Y_gt, 'float32')
    Y_pred = tf.cast(Y_pred, 'float32')
    w = tf.einsum("bwhc->bc", Y_gt)
    w = 1 / ((w + 1e-10) ** 2)
    intersection = w * tf.einsum("bwhc,bwhc->bc", Y_pred, Y_gt)
    union = w * (tf.einsum("bwhc->bc", Y_pred) + tf.einsum("bwhc->bc", Y_gt))

    divided = 1 - 2 * (tf.einsum("bc->b", intersection) + 1e-10) / (tf.einsum("bc->b", union) + 1e-10)

    loss = tf.reduce_mean(divided)
    return loss


def generalised_dice_loss_2d(Y_gt, Y_pred):
    smooth = 1e-5
    w = tf.reduce_sum(Y_gt, axis=[1, 2])
    w = 1 / (w ** 2 + smooth)

    numerator = Y_gt * Y_pred
    numerator = w * tf.reduce_sum(numerator, axis=[1, 2])
    numerator = tf.reduce_sum(numerator, axis=1)

    denominator = Y_pred + Y_gt
    denominator = w * tf.reduce_sum(denominator, axis=[1, 2])
    denominator = tf.reduce_sum(denominator, axis=1)

    gen_dice_coef = 2 * numerator / (denominator + smooth)
    loss = tf.reduce_mean(1 - gen_dice_coef)
    return loss


def surface_loss_3d(Y_gt, Y_pred):
    multipled = tf.reduce_sum(Y_gt * Y_pred, axis=[0,1, 2, 3, 4])
    loss = tf.reduce_mean(multipled)
    return loss


def surface_loss_2d(Y_gt, Y_pred):
    multipled = tf.reduce_sum(Y_gt * Y_pred, axis=[0,1, 2, 3])
    loss = tf.reduce_mean(multipled)
    return loss


def focal_loss_sigmodv1(Y_gt, Y_pred,alpha=0.25, gamma=2):
   epsilon = 1e-5
   pt_1 = tf.where(tf.equal(Y_gt, 1), Y_pred, tf.ones_like(Y_pred))
   pt_0 = tf.where(tf.equal(Y_gt, 0), Y_pred, tf.zeros_like(Y_pred))
   # clip to prevent NaN's and Inf's
   pt_1 = tf.clip_by_value(pt_1, epsilon, 1. - epsilon)
   pt_0 = tf.clip_by_value(pt_0, epsilon, 1. - epsilon)
   loss_1 = -alpha * tf.pow(1. - pt_1, gamma) * tf.log(pt_1)
   loss_0 = -(1 - alpha) * tf.pow(pt_0, gamma) * tf.log(1. - pt_0)
   loss = tf.reduce_sum(loss_1 + loss_0)
   return loss




x = tf.constant(value=5., shape=(3, 32, 32, 32, 1))
y = tf.constant(value=1., shape=(3, 32, 32, 32, 1))
surface_loss_3d(x, y)

In [ ]:
tf.keras.datasets.cifar100

In [ ]:
#phisycal_devices = tf.config.experimental.list_physical_devices('GPU')
#print("Num of GPUs available: ",len(phisycal_devices) )

# tf.config.experimental.set_memory_growth(phisycal_devices[0] , True)

In [ ]:
drive.mount('/content/drive',force_remount=True)

In [ ]:
import numpy as np

In [ ]:
X = np.load('/content/drive/MyDrive/segment_data.npy')
Y = np.load('/content/drive/MyDrive/segment_labels.npy')

In [ ]:
Y = (Y/255.0)[:700]
X = (X/255.0)[:700]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)
print(X_train.shape)
print(X_test.shape)

In [ ]:


%%time
seed=24

img_data_gen_args = dict(rotation_range=90,
                     width_shift_range=0.3,
                     height_shift_range=0.3,
                     horizontal_flip=True,
                     vertical_flip=True,
                     shear_range=0.5,
                     fill_mode='reflect')

mask_data_gen_args = dict(rotation_range=90,
                     width_shift_range=0.3,
                     height_shift_range=0.3,
                     horizontal_flip=True,
                     vertical_flip=True,
                     shear_range=0.5,
                     fill_mode='reflect') #Binarize the output again.
X_train=X_train.reshape((525, 256, 256, 3))
X_test=X_test.reshape((175, 256, 256, 3))
image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_data_generator.fit(X_train, augment=True, seed=seed)
image_generator = image_data_generator.flow(tf.convert_to_tensor(X_train), seed=seed)
valid_img_generator = image_data_generator.flow(tf.convert_to_tensor(X_test), seed=seed)

In [ ]:
%%time
y_train = y_train.reshape((525, 256, 256,3))
y_test = y_test.reshape((175, 256, 256,3))

mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
mask_data_generator.fit(y_train, augment=True, seed=seed)
mask_generator = mask_data_generator.flow(tf.convert_to_tensor(y_train), seed=seed)
valid_mask_generator = mask_data_generator.flow(tf.convert_to_tensor(y_test), seed=seed)

In [ ]:
def my_image_mask_generator(image_generator, mask_generator):
    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

my_generator = my_image_mask_generator(image_generator, mask_generator)

validation_datagen = my_image_mask_generator(valid_img_generator, valid_mask_generator)

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 3

In [ ]:
kernel_initializer =  'he_normal'
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
    c1 = Dropout(0.1)(c1)

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    model.summary()

    return model
def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = get_model()

In [ ]:
def conv_block(inputs, filter_count, pool=True, batchnorm = True):

    #first layer
    x = Conv2D(filter_count, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #second layer
    x = Conv2D(filter_count, 3, padding = 'same', kernel_initializer = 'he_normal')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x



def deconv_block(inputs, concat_layer, filter_count, pool = False):
    u = UpSampling2D((2,2), interpolation='bilinear')(inputs)
    c = Concatenate()([u, concat_layer])
    x = conv_block(c, filter_count, pool = pool, batchnorm = True)
    return u, c, x

def make_me_a_unet(shape, num_classes):

    inputs = Input(shape) # 768 x 1152

    # Downsampling side of the UNET i.e. the encoder !

    x1, p1 = conv_block(inputs, 32, pool=True, batchnorm=True)
    x2, p2 = conv_block(p1, 64, pool=True, batchnorm=True)
    x3, p3 = conv_block(p2, 96, pool=True, batchnorm=True)
    x4, p4 = conv_block(p3, 128, pool=True, batchnorm=True)
    b = conv_block(p4, 256, pool=False, batchnorm=True)

    # Upsampling side of the UNET i.e the decoder !

    u1, c1, x5 = deconv_block(b, x4, 128)
    u2, c2, x6 = deconv_block(x5, x3, 96)
    u3, c3, x7 = deconv_block(x6, x2, 64)
    u4, c4, x8 = deconv_block(x7, x1, 32)

    # The output layer

    output = Conv2D(num_classes, 1, padding='same', activation='softmax')(x8)

    #softmax for multiclass classification, num_classes = 23 !

    return Model(inputs, output)



model = make_me_a_unet((IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS), 3)

In [ ]:
model.summary()

In [ ]:
Y.shape

In [ ]:
from tensorflow import keras
#@title Select parameters {run: "auto"}

optimizer = 'adam' #@param ["adam" , "momentum" , "rmsprop" , "adagrad", "nag"] {type :"string"}
Learning_rate = 0.0001 #@param {type:"number"}
LR = float(Learning_rate)
if (optimizer=="momentum"):
  opt = keras.optimizers.SGD(learning_rate=LR,momentum=0.9)
if (optimizer=="adam"):
  opt = keras.optimizers.Adam(learning_rate=LR)
if (optimizer=="RMSprop"):
  opt = keras.optimizers.RMSProp(learning_rate=LR)
if (optimizer=="Adagrad"):
  opt = keras.optimizers.adagrad(learning_rate=LR)
if (optimizer=="nag"):
  opt = keras.optimizers.SGD(learning_rate=LR,momentum=0.9,nesterov=True)

batch_size = 64
steps_per_epoch = 3*(len(X_train))//batch_size
model.compile(optimizer= opt, loss = 'binary_crossentropy', metrics=['acc'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
history = model.fit(my_generator,
                    validation_data = validation_datagen,
                    steps_per_epoch = steps_per_epoch,
                    validation_steps = steps_per_epoch,
                    epochs=200,
                    callbacks=callback)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
# test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img, 0)
print(test_img_number)
prediction = model.predict(test_img_input)
# prediction = prediction> threshold_otsu(prediction)
prediction=prediction[0,:,:,0]
plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction, cmap='gray')

plt.show()


In [ ]:
pred, y_pred = [model.predict(np.expand_dims(xt,0)) for xt in list(X_test)], []
for p in pred:
  t = p[p>0.1].mean()
  p[p<t],p[p>=t] = 0, 1
  y_pred.append(p)

In [ ]:
y_test_metrics = np.array([np.ravel(yt) for yt in y_test])
y_pred_metrics = np.array([np.ravel(yp) for yp in y_pred])

In [ ]:
plt.imshow(np.squeeze(y_pred[20]),cmap='binary')
# np.unique(y_pred[5])

In [ ]:
plt.imshow(np.squeeze(y_test[20]),cmap='binary')

In [ ]:
def recall_m(y_train, y_pred):
      true_positives = K.sum(K.round(K.clip(y_train * y_pred, 0, 1)))
      possible_positives = K.sum(K.round(K.clip(y_train, 0, 1)))
      recall = true_positives / (possible_positives + K.epsilon())
      return recall

def precision_m(y_train, y_pred):
    true_positives = K.sum(K.round(K.clip(y_train * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_train, y_pred):
    precision = precision_m(y_train, y_pred)
    recall = recall_m(y_train, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def mcc(y_true, y_pred):
    y_true_pos = K.round(K.clip(y_true, 0, 1))
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    true_pos = K.sum(y_true_pos * y_pred_pos)
    true_neg = K.sum((1 - y_true_pos) * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))

    numerator = (true_pos * true_neg - false_pos * false_neg)
    denominator = K.sqrt((true_pos + false_pos) * (true_pos + false_neg) * (true_neg + false_pos) * (true_neg + false_neg))

    mcc = numerator / (denominator + K.epsilon())
    return mcc


In [ ]:
precision_m(y_test_metrics.astype('float32'),y_pred_metrics.astype('float32')),recall_m(y_test_metrics.astype('float32'),y_pred_metrics.astype('float32')),f1_m(y_test_metrics.astype('float32'),y_pred_metrics.astype('float32'))

In [ ]:
mcc(y_test_metrics.astype('float32'),y_pred_metrics.astype('float32'))